<a href="https://colab.research.google.com/github/aomiglionni/datascience_python/blob/master/P4DS4D%3B_19%3B_Increasing_Complexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_boston
from random import shuffle
boston = load_boston()
seed(0) # Creates a replicable shuffling
new_index = range(boston.data.shape[0])
shuffle(new_index) # shuffling the index
X, y = boston.data[new_index],
boston.target[new_index]
print X.shape, y.shape

In [ ]:
import pandas as pd
df = pd.DataFrame(X,columns=boston.feature_names)
df['target'] = y

In [ ]:
scatter = df.plot(kind='scatter',
x='LSTAT', y='target', c='r')

In [ ]:
import numpy as np
from
sklearn.feature_selection.univariate_selec
tion import f_regression
F, pval = f_regression(df['LSTAT'],y)
print 'F score for the original feature
%.1f' % F
F, pval =
f_regression(np.log(df['LSTAT']),y)
print 'F score for the transformed feature
%.1f' % F

In [ ]:
from sklearn.linear_model import
LinearRegression
from sklearn.cross_validation import
cross_val_score
from sklearn.cross_validation import KFold
regression =
LinearRegression(normalize=True)
crossvalidation = KFold(n=X.shape[0],
n_folds=10, shuffle=True, random_state=1)

SLIDE 22

In [ ]:
df =
pd.DataFrame(X,columns=boston.feature_name
s)
baseline =
np.mean(cross_val_score(regression, df, y,
scoring='r2', cv=crossvalidation,
  n_jobs=1))
interactions = list()
for feature_A in boston.feature_names:
  for feature_B in boston.feature_names:
    if feature_A > feature_B:
      df['interaction'] = df[feature_A]
* df[feature_B]
      score =
np.mean(cross_val_score(regression, df, y,
scoring='r2',
        cv=crossvalidation, n_jobs=1))
    if score > baseline:
          interactions.append((feature
_A, feature_B, round(score,3)))
print 'Baseline R2: %.3f' % baseline
print 'Top 10 interactions: %s' %
sorted(interactions, key=lambda(x):x[2],
reverse=True)[:10]

SLIDE 24


In [ ]:
colors = ['k' if v > np.mean(y) else 'w'
for v in y]
scatter = df.plot(kind='scatter', x='RM',
y='LSTAT', c=colors)

SLIDE 27

In [ ]:
polyX =
pd.DataFrame(X,columns=boston.feature_name
s)
baseline =
np.mean(cross_val_score(regression, polyX,
y,
        scoring='mean_squared_error',
  cv=crossvalidation, n_jobs=1))
improvements = [baseline]
for feature_A in boston.feature_names:
  polyX[feature_A+'^2'] =
polyX[feature_A]**2
  improvements.append(np.mean(cross_val_s
core(regression, polyX, y,
  scoring='mean_squared_error',
cv=crossvalidation, n_jobs=1)))
  for feature_B in boston.feature_names:
    if feature_A > feature_B:
      polyX[feature_A+'*'+feature_B] =
polyX[feature_A] * polyX[feature_B]
      improvements.append(np.mean(cross
_val_score(regression, polyX, y,
      scoring='mean_squared_error',
cv=crossvalidation, n_jobs=1)))

SLIDE 30

In [ ]:
print shape(polyX)
crossvalidation = KFold(n=X.shape[0],
n_folds=10, shuffle=True, random_state=1)
print 'Mean squared error %.3f' %
abs(np.mean(cross_val_score(regression,
polyX, y,
  scoring='mean_squared_error',
cv=crossvalidation, n_jobs=1)))


SLIDE 37

In [ ]:
from sklearn.grid_search import
GridSearchCV
from sklearn.linear_model import Ridge
ridge = Ridge(normalize=True)
search = GridSearchCV(estimator=ridge,
param_grid={'alpha':np.logspace(-5,2,8)},
                  scoring='mean_squared_er
ror', n_jobs=1, refit=True, cv=10)
search.fit(polyX,y)
print 'Best parameters: %s' %
search.best_params_
print 'CV MSE of best parameters: %.3f' %
abs(search.best_score_)

SLIDE 40

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(normalize=True)
search = GridSearchCV(estimator=lasso,
param_grid={'alpha':np.logspace(-5,2,8)},
                    scoring='mean_squared
_error', n_jobs=1, refit=True, cv=10)
search.fit(polyX,y)
print 'Best parameters: %s' %
search.best_params_
print 'CV MSE of best parameters: %.3f' %
abs(search.best_score_)


SLIDE 42

In [ ]:
lasso = Lasso(normalize=True, alpha=0.01)
lasso.fit(polyX,y)
print
polyX.columns[np.abs(lasso.coef_)>0.0001].
values

['CRIM*CHAS' 'ZN*CRIM' 'ZN*CHAS'
'INDUS*DIS' 'CHAS*B' 'NOX^2' 'NOX*DIS'
'RM^2' 'RM*CRIM' 'RM*NOX' 'RM*PTRATIO'
'RM*B' 'RM*LSTAT' 'RAD*B' 'TAX*DIS'
'PTRATIO*NOX' 'LSTAT^2']


SLIDE 45

In [ ]:
from sklearn.linear_model import
ElasticNet
elastic = ElasticNet(normalize=True)
search = GridSearchCV(estimator=elastic,
param_grid={'alpha':np.logspace(-5,2,8),
  'l1_ratio': [0.25, 0.5, 0.75]},
  scoring='mean_squared_error', n_jobs=1,
refit=True, cv=10)
search.fit(polyX,y)
print 'Best parameters: %s' %
search.best_params_
print 'CV MSE of best parameters: %.3f' %
abs(search.best_score_)

SLIDE 52

In [ ]:
from sklearn.linear_model import
SGDRegressor
from sklearn.preprocessing import
StandardScaler
SGD = SGDRegressor(loss='squared_loss',
penalty='l2', alpha=0.0001,
              l1_ratio=0.15,
              n_iter=2000)
scaling = StandardScaler()
scaling.fit(polyX)
scaled_X = scaling.transform(polyX)
print 'CV MSE: %.3f' %
abs(np.mean(cross_val_score(SGD, scaled_X,
y,
  scoring='mean_squared_error',
cv=crossvalidation, n_jobs=1)))

SLIDE 54

In [ ]:
from sklearn.metrics import
mean_squared_error
from sklearn.cross_validation import
train_test_split
X_train, X_test, y_train, y_test =
train_test_split(scaled_X, y,
test_size=0.20,
  random_state=2)
SGD = SGDRegressor(loss='squared_loss',
penalty='l2', alpha=0.0001, l1_ratio=0.15)
improvements = list()
for z in range(1000):
  SGD.partial_fit(X_train, y_train)
  improvements.append(mean_squared_error(
y_test, SGD.predict(X_test)))


SLIDE 56

In [ ]:
import matplotlib.pyplot as plt
plt.subplot(1,2,1)
plt.plot(range(1,11),np.abs(improvements[:
10]),'o--')
plt.xlabel('Partial fit initial
iterations')
plt.ylabel('Test set mean squared error')
plt.subplot(1,2,2)
plt.plot(range(100,1000,100),np.abs(improv
ements[100:1000:100]),'o--')
plt.xlabel('Partial fit ending
iterations')
plt.show()

SLIDE 81

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
X,y = digits.data, digits.target

SLIDE 83

In [ ]:
import matplotlib.pyplot as plt
for k,img in enumerate(range(10)):
  plt.subplot(2, 5, k)
  plt.imshow(digits.images[img],
  cmap='binary',interpolation='none')
plt.show()

SLIDE 89

In [ ]:
from sklearn.cross_validation import
train_test_split, cross_val_score
from sklearn.preprocessing import
MinMaxScaler
# We keep 30% random examples for test
X_train, X_test, y_train, y_test =
train_test_split(X, y, test_size=0.3,
random_state=101)

SLIDE 90

In [ ]:
# We scale the data in the range [-1,1]
scaling = MinMaxScaler(feature_range=(-1,
1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

SLIDE 92

In [ ]:
from sklearn.svm import SVC
# We balance the clasess so you can see
how it works
learning_algo = SVC(kernel='linear',
class_weight='auto')

SLIDE 93

In [ ]:
cv_performance =
cross_val_score(learning_algo, X_train,
y_train, cv=10)
test_performance =
learning_algo.fit(X_train,
y_train).score(X_test, y_test)

SLIDE 94

In [ ]:
print 'Cross-validation accuracy score:
%0.3f,
test accuracy score: %0.3f' %
(np.mean(cv_performance),test_performance)

SLIDE 96

In [ ]:
from sklearn.grid_search import
GridSearchCV
learning_algo = SVC(kernel='linear',
class_weight='auto', random_state=101)
search_space = {'C': np.logspace(-3, 3,
7)}
gridsearch = GridSearchCV(learning_algo,
param_grid=search_space,
scoring='accuracy',
refit=True, cv=10)
gridsearch.fit(X_train,y_train)

SLIDE 99

In [ ]:
cv_performance = gridsearch.best_score_
test_performance =
gridsearch.score(X_test, y_test)

SLIDE 100

In [ ]:
print 'Cross-validation accuracy score:
%0.3f,
test accuracy score: %0.3f' %
(cv_performance,test_performance)
print 'Best C parameter: %0.1f' %
gridsearch.best_params_['C']

SLIDE 104

In [ ]:
from sklearn.grid_search import
GridSearchCV
learning_algo = SVC(class_weight='auto',
random_state=101)
search_space = [{'kernel': ['linear'],
'C': np.logspace(-3, 3, 7)},
{'kernel': ['rbf'],'degree':
[2,3,4],'C':np.logspace(-3, 3, 7),
'gamma': np.logspace(-3, 2,
6)}]
gridsearch = GridSearchCV(learning_algo,
param_grid=search_space,
scoring='accuracy',
refit=True, cv=10)
gridsearch.fit(X_train,y_train)
cv_performance = gridsearch.best_score_
test_performance =
gridsearch.score(X_test, y_test)
print 'Cross-validation accuracy score:
%0.3f,
test accuracy score: %0.3f' %
(cv_performance,test_performance)
print 'Best parameters: %s' %
gridsearch.best_params_

SLIDE 108

In [ ]:
from sklearn import datasets
boston = datasets.load_boston()
X,y = boston.data, boston.target
X_train, X_test, y_train, y_test =
train_test_split(X, y, test_size=0.3,
  random_state=101)
scaling = MinMaxScaler(feature_range=(-1,
1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

SLIDE 111

In [ ]:
from sklearn.svm import SVR
learning_algo = SVR(random_state=101)
search_space = [{'kernel': ['linear'],
'C': np.logspace(-3, 2, 6),'epsilon': [0,
0.01,
0.1, 0.5, 1, 2, 4]},{'kernel': ['rbf'],
'degree':[2,3],'C':np.logspace(-3, 3, 7),
'gamma': np.logspace(-3, 2, 6),
'epsilon': [0, 0.01, 0.1, 0.5, 1, 2, 4]}]
gridsearch = GridSearchCV(learning_algo,
param_grid=search_space, refit=True,
  scoring= 'r2', cv=10)
gridsearch.fit(X_train,y_train)
cv_performance = gridsearch.best_score_
test_performance =
gridsearch.score(X_test, y_test)
print 'Cross-validation R2 score: %0.3f,
  test R2 score: %0.3f' %
(cv_performance,test_performance)
print 'Best parameters: %s' %
gridsearch.best_params_
Cross-validation R2 score: 0.833, test R2
score: 0.871
Best parameters: {'epsilon': 2,'C':
1000.0,'gamma': 0.10000000000000001,
'degree': 2,'kernel': 'rbf'}

SLIDE 117

In [ ]:
from sklearn.datasets import
make_classification
X,y = make_classification(n_samples=10**4,
n_features=15, n_informative=10,
  random_state=101)
X_train, X_test, y_train, y_test =
train_test_split(X, y, test_size=0.3,
  random_state=101)
from sklearn.svm import SVC, LinearSVC
slow_SVM = SVC(kernel="linear",
random_state=101)
fast_SVM = LinearSVC(random_state=101,
penalty='l2', loss='l1')
slow_SVM.fit(X_train, y_train)
fast_SVM.fit(X_train, y_train)
print 'SVC test accuracy score: %0.3f' %
slow_SVM.score(X_test, y_test)
print 'LinearSVC test accuracy score:
%0.3f' % fast_SVM.score(X_test, y_test)


SLIDE 119

In [ ]:
import timeit
X,y = make_classification(n_samples=10**4,
n_features=15, n_informative=10,
  random_state=101)
print 'avg secs for SVC, best of 3: %0.1f'
% np.mean(timeit.timeit("slow_SVM.fit(X,
y)",
  "from __main__ import slow_SVM, X, y",
number=1))
print 'avg secs for LinearSVC, best of 3:
%0.1f' % np.mean(
  timeit.timeit("fast_SVM.fit(X, y)",
"from __main__ import fast_SVM, X, y",
number=1))


SLIDE 123

In [ ]:
from sklearn.linear_model import
SGDClassifier
X,y = make_classification(n_samples=10**6,
n_features=15, n_informative=10,
  random_state=101)
X_train, X_test, y_train, y_test =
train_test_split(X, y, test_size=0.3,
  random_state=101) 

SLIDE 125

In [ ]:
fast_SVM = LinearSVC(random_state=101)
fast_SVM.fit(X_train, y_train)
print 'LinearSVC test accuracy score:
%0.3f' % fast_SVM.score(X_test, y_test)
print 'avg secs for LinearSVC, best of 3:
%0.1f' % np.mean(
  timeit.timeit("fast_SVM.fit(X_train,
y_train)",
  "from __main__ import fast_SVM,
X_train, y_train", number=1))

SLIDE 127

In [ ]:
stochastic_SVM =
SGDClassifier(loss='hinge', n_iter=5,
shuffle=True, random_state=101)
stochastic_SVM.fit(X_train, y_train)
print 'SGDClassifier test accuracy score:
%0.3f' % stochastic_SVM.score(X_test,
y_test)
print 'avg secs for SGDClassifier, best of
3: %0.1f' % np.mean(
  timeit.timeit("stochastic_SVM.fit(X_tra
in, y_train)",
  "from __main__ import stochastic_SVM,
X_train, y_train", number=1))